# GPS Project 2
Team Member: __Atreish Ramlakhan, Nosson Weissman, Xiaolan Li__

In [1]:
# import libararies
import numpy as np
from itertools import combinations
import math
from numpy import linalg as LA

# Question 1

In [2]:
# get Reciever Position by satellites position and tansmission time
def RecieverPosition(Pos_SatelsMatrix,Transmission_t,InitialVector=np.array([0, 0, 6370, 0]) ,N=10):
    A = Pos_SatelsMatrix[:,0]
    B = Pos_SatelsMatrix[:,1]
    C = Pos_SatelsMatrix[:,2]
    CurrentVector = InitialVector
    c = 299792.458
    
    F = np.zeros(4)
    DF = np.zeros((4,4))
    for i in range(N): 
        x = CurrentVector[0]
        y = CurrentVector[1]
        z = CurrentVector[2]
        d = CurrentVector[3]
        #print('CurrentRecieverPosition',CurrentVector)
        
         # functions of (x,y,z,d)
        for i in range(len(InitialVector)):
            F[i] = ((x - A[i])**2 + (y-B[i])**2 + (z-C[i])**2 - (c*(Transmission_t[i] - d))**2)
            DF[i][0] = (2*(x-A[i]))
            DF[i][1] = (2*(y-B[i]))
            DF[i][2] = (2*(z-C[i]))
            DF[i][3] = (2*c**2*(Transmission_t[i]-d))
        # calculate with Multivariate Newton’s Method
        # Formula: X(i+1) = Xi- DF(Xi)^(-1)*F(Xi)
        CurrentVector = CurrentVector - np.dot(np.linalg.inv(DF),F)
        
    return CurrentVector

In [3]:
# solve problem 1

# define constant value satellites position and transmission time
Pos_SatelsMatrix = np.array([[15600, 7540, 20140], [18760, 2750, 18610], [17610, 14630, 13480], [19170, 610, 18390]])
Transmission_t = np.array([0.07074, 0.07220, 0.07690, 0.07242])
InitialVector = np.array([0, 0, 6370, 0])

# get Reciever Position with given constant value
RecieverPos = RecieverPosition(Pos_SatelsMatrix,Transmission_t,InitialVector)
print('RecieverPosition',RecieverPos)

RecieverPosition [-4.17727096e+01 -1.67891941e+01  6.37005956e+03 -3.20156583e-03]


# Question 2

Define functions

In [4]:
# satellite positions in km and travel times in seconds
def SatellitePosition(phi,theta):
    Pos_SatellMatrix = np.zeros((4,3))
    Transmission_t = np.zeros(4)
    c = 299792.458
    rho=26570;
    for i in range(4):
        # Pos_SatellMatrix[i][0] mean A array
        # Pos_SatellMatrix[i][1] mean B array
        # Pos_SatellMatrix[i][2] mean C array
        Pos_SatellMatrix[i][0]=rho*math.cos(phi[i])*math.cos(theta[i])
        Pos_SatellMatrix[i][1]=rho*math.cos(phi[i])*math.sin(theta[i])
        Pos_SatellMatrix[i][2]=rho*math.sin(phi[i])
        R =math.sqrt(Pos_SatellMatrix[i][0]**2+Pos_SatellMatrix[i][1]**2+(Pos_SatellMatrix[i][2]-6370)**2)
        Transmission_t[i]=0.0001+R/c
    return Pos_SatellMatrix, Transmission_t

In [5]:
# changes in the transmission time (+/- 10^-8)
def ChangeTranT(t0, N = 15):
    NewTranT = np.zeros((15,4))
    dt=10**(-8);
    NewTranT[0]=t0;
    NewTranT[1]=[t0[0]-dt, t0[1]+dt, t0[2]+dt, t0[3]+dt];
    NewTranT[2]=[t0[0]+dt, t0[1]-dt, t0[2]+dt, t0[3]+dt];
    NewTranT[3]=[t0[0]+dt, t0[1]+dt, t0[2]-dt, t0[3]+dt];
    NewTranT[4]=[t0[0]+dt, t0[1]+dt, t0[2]+dt, t0[3]-dt];
    NewTranT[5]=[t0[0]-dt, t0[1]-dt, t0[2]+dt, t0[3]+dt];
    NewTranT[6]=[t0[0]+dt, t0[1]+dt, t0[2]-dt, t0[3]-dt];
    NewTranT[7]=[t0[0]-dt, t0[1]+dt, t0[2]+dt, t0[3]-dt];
    NewTranT[8]=[t0[0]+dt, t0[1]-dt, t0[2]-dt, t0[3]+dt];
    NewTranT[9]=[t0[0]-dt, t0[1]+dt, t0[2]-dt, t0[3]+dt];
    NewTranT[10]=[t0[0]+dt, t0[1]-dt, t0[2]+dt, t0[3]-dt];
    NewTranT[11]=[t0[0]+dt, t0[1]-dt, t0[2]-dt, t0[3]-dt];
    NewTranT[12]=[t0[0]-dt, t0[1]+dt, t0[2]-dt, t0[3]-dt];
    NewTranT[13]=[t0[0]-dt, t0[1]-dt, t0[2]+dt, t0[3]-dt];
    NewTranT[14]=[t0[0]-dt, t0[1]-dt, t0[2]-dt, t0[3]+dt];
    return NewTranT

In [6]:
# calculate emf
def Cal_emf(RecieverPos,RecieverPosMatrix):
    c = 299792.458
    dt=np.array([10**(-8),10**(-8),10**(-8),10**(-8)])
    du=RecieverPos-RecieverPosMatrix[0]
    normdu = LA.norm(du, np.inf)   #the forward error (or output error)
    normdt = c*LA.norm(dt, np.inf)   #the backward error (or input error)    
    return (normdu/normdt)

In [7]:
# solve problem 2

# define phi and theta
phi=[math.pi/8, 2*math.pi/8, 3*math.pi/8, 4*math.pi/8]
theta=[math.pi/2, math.pi, 3*math.pi/2, 2*math.pi]

# define solution lists
RecieverPosMatrix = []
emf = []

#call functions to get satellites position and different tansmission time
Pos_SatelsMatrix, Transmission_t = SatellitePosition(phi,theta)
NewTranT =  ChangeTranT(Transmission_t)

# get Reciever Position in different tansmission time and return emf
for t_vector in NewTranT:
    InitialVector = np.array([0, 0, 6370, 0])
    RecieverPos = RecieverPosition(Pos_SatelsMatrix,t_vector,InitialVector,N =20)
    RecieverPosMatrix.append(RecieverPos)
    emf.append(Cal_emf(RecieverPos,RecieverPosMatrix))
print('RecieverPosMatrix',RecieverPosMatrix)
print('emf',emf)
print('Max_emf',np.max(emf))

    

RecieverPosMatrix [array([1.08231226e-12, 2.91975686e-12, 6.37000000e+03, 1.00000000e-04]), array([2.91857713e-03, 1.41483329e-03, 6.36999457e+03, 9.99918958e-05]), array([-7.18715868e-03,  3.94331383e-12,  6.37000000e+03,  1.00010000e-04]), array([ 5.90815207e-03, -9.41788180e-03,  6.36998901e+03,  9.99733512e-05]), array([-1.63956590e-03,  8.00303975e-03,  6.37001641e+03,  1.00044753e-04]), array([-4.26858328e-03,  1.41483329e-03,  6.36999457e+03,  9.99918958e-05]), array([ 4.26858230e-03, -1.41483301e-03,  6.37000543e+03,  1.00008104e-04]), array([1.27900957e-03, 9.41787458e-03, 6.37001099e+03, 1.00026649e-04]), array([-1.27901013e-03, -9.41788179e-03,  6.36998901e+03,  9.99733512e-05]), array([ 8.82673204e-03, -8.00305033e-03,  6.36998359e+03,  9.99552470e-05]), array([-8.82672124e-03,  8.00303975e-03,  6.37001641e+03,  1.00044753e-04]), array([-2.91857657e-03, -1.41483301e-03,  6.37000543e+03,  1.00008104e-04]), array([ 7.18716060e-03, -3.80098577e-13,  6.37000000e+03,  9.99900000

# Question 3

In [8]:
# solve problem 3
# different phi and theta range
phi=[math.pi/8+0.01*math.pi/2, math.pi/8+0.02*math.pi/2, math.pi/8+0.03*math.pi/2, math.pi/8+0.04*math.pi/2];
theta=[math.pi/2+0.01*2*math.pi, math.pi/2+0.02*2*math.pi, math.pi/2+0.03*2*math.pi, math.pi/2+0.04*2*math.pi];

# define solution lists
RecieverPosMatrix = []
emf = []

#call functions to get satellites position and different tansmission time
Pos_SatelsMatrix, Transmission_t = SatellitePosition(phi,theta)
NewTranT =  ChangeTranT(Transmission_t)

# get Reciever Position in different tansmission time and return emf
for t_vector in NewTranT:
    InitialVector = np.array([0, 0, 6370, 0])
    RecieverPos = RecieverPosition(Pos_SatelsMatrix,t_vector,InitialVector,N =20)
    RecieverPosMatrix.append(RecieverPos)
    emf.append(Cal_emf(RecieverPos,RecieverPosMatrix))
print('RecieverPosMatrix',RecieverPosMatrix)
print('emf',emf)
print('Max_emf',np.max(emf))

RecieverPosMatrix [array([5.90434258e-09, 4.56794012e-10, 6.37000000e+03, 1.00000000e-04]), array([-2.03962310e+01, -6.32482317e+00,  6.28893939e+03,  3.72909005e-05]), array([6.14019752e+01, 1.75317500e+01, 6.61287011e+03, 2.77744273e-04]), array([-6.20662121e+01, -1.62324256e+01,  6.12628578e+03, -8.19405632e-05]), array([2.07627335e+01, 4.93378346e+00, 6.45139615e+03, 1.57351150e-04]), array([4.10942393e+01, 1.12363777e+01, 6.53196157e+03, 2.17892542e-04]), array([-4.12132180e+01, -1.12730329e+01,  6.20783539e+03, -2.17152302e-05]), array([ 3.96330641e-01, -1.38188751e+00,  6.37038657e+03,  9.55919422e-05]), array([-3.96261548e-01,  1.38174619e+00,  6.36961358e+03,  1.04407600e-04]), array([-8.25568035e+01, -2.25872101e+01,  6.04505134e+03, -1.47546459e-04]), array([8.20802920e+01, 2.24404485e+01, 6.69413458e+03, 3.32219607e-04]), array([2.03657977e+01, 6.31435234e+00, 6.45100573e+03, 1.61750301e-04]), array([-6.16718026e+01, -1.76180783e+01,  6.12666056e+03, -8.63549515e-05]), arra